# Day 5 Lab 1: SageMaker Setup & Data Preparation
## SecureBank Customer Churn Prediction

**Objective:** Set up SageMaker environment and prepare banking data for ML training

**What You'll Learn:**
- Configure SageMaker session and S3 buckets
- Load and explore banking customer data
- Perform data preprocessing and feature engineering
- Split data into training and validation sets
- Upload prepared data to S3

## Step 1: Import Libraries and Initialize SageMaker

In [ ]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize SageMaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'securebank/churn-prediction'
role = get_execution_role()

print(f"SageMaker Session initialized")
print(f"S3 Bucket: {bucket}")
print(f"IAM Role: {role}")

## Step 2: Load Sample Banking Data

We'll use a sample telecom churn dataset as a proxy for banking customer churn.

In [ ]:
# Generate sample churn dataset (banking customers)
import numpy as np

print("📊 Generating sample banking customer churn dataset...\n")

np.random.seed(42)
n_customers = 5000

# Generate realistic banking customer data
df = pd.DataFrame({
    'State': np.random.choice(['CA', 'NY', 'TX', 'FL', 'IL'], n_customers),
    'Account Length': np.random.randint(1, 250, n_customers),
    'Area Code': np.random.choice([408, 415, 510], n_customers),
    'Phone': [f'{np.random.randint(100,999)}-{np.random.randint(1000,9999)}' for _ in range(n_customers)],
    'Int\'l Plan': np.random.choice(['yes', 'no'], n_customers, p=[0.1, 0.9]),
    'VMail Plan': np.random.choice(['yes', 'no'], n_customers, p=[0.3, 0.7]),
    'VMail Message': np.random.poisson(8, n_customers),
    'Day Mins': np.random.normal(180, 50, n_customers).clip(0, 400),
    'Day Calls': np.random.poisson(100, n_customers),
    'Day Charge': np.random.normal(30, 8, n_customers).clip(0, 70),
    'Eve Mins': np.random.normal(200, 50, n_customers).clip(0, 400),
    'Eve Calls': np.random.poisson(100, n_customers),
    'Eve Charge': np.random.normal(17, 4, n_customers).clip(0, 35),
    'Night Mins': np.random.normal(200, 50, n_customers).clip(0, 400),
    'Night Calls': np.random.poisson(100, n_customers),
    'Night Charge': np.random.normal(9, 2, n_customers).clip(0, 18),
    'Intl Mins': np.random.normal(10, 3, n_customers).clip(0, 20),
    'Intl Calls': np.random.poisson(4, n_customers),
    'Intl Charge': np.random.normal(2.7, 0.8, n_customers).clip(0, 6),
    'CustServ Calls': np.random.poisson(1.5, n_customers),
    'Churn?': np.random.choice(['True.', 'False.'], n_customers, p=[0.14, 0.86])
})

# Save to file for consistency with original lab
df.to_csv('churn.txt', index=False)

print(f"✅ Generated {len(df)} customer records")
print(f"Dataset shape: {df.shape}")
print(f"Churn rate: {(df['Churn?'] == 'True.').mean():.1%}")
print(f"\nFirst few rows:")
df.head()

## Step 3: Exploratory Data Analysis

In [ ]:
# Check data info
print("Dataset Information:")
df.info()

print("\nStatistical Summary:")
df.describe()

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Check target distribution
print("\nChurn Distribution:")
print(df['Churn?'].value_counts())
print(f"\nChurn Rate: {df['Churn?'].value_counts(normalize=True)['True.']:.2%}")

## Step 4: Data Preprocessing

In [ ]:
# Convert target to binary (1 for churn, 0 for no churn)
df['Churn'] = (df['Churn?'] == 'True.').astype(int)

# Drop original churn column and phone number
df = df.drop(['Churn?', 'Phone'], axis=1)

# Convert categorical variables to dummy variables
df = pd.get_dummies(df)

# Move target column to first position (XGBoost requirement)
cols = df.columns.tolist()
cols = ['Churn'] + [c for c in cols if c != 'Churn']
df = df[cols]

print(f"Preprocessed data shape: {df.shape}")
print(f"\nFeatures: {df.columns.tolist()[:10]}...")
df.head()

## Step 5: Train/Validation Split

In [ ]:
# Split data: 80% train, 20% validation
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Churn'])

print(f"Training set: {train_data.shape}")
print(f"Validation set: {val_data.shape}")
print(f"\nTraining set churn rate: {train_data['Churn'].mean():.2%}")
print(f"Validation set churn rate: {val_data['Churn'].mean():.2%}")

## Step 6: Upload Data to S3

In [ ]:
# Save to CSV (no header, no index - XGBoost format)
train_data.to_csv('train.csv', header=False, index=False)
val_data.to_csv('validation.csv', header=False, index=False)

# Upload to S3
train_s3_path = sess.upload_data('train.csv', bucket=bucket, key_prefix=f'{prefix}/data/train')
val_s3_path = sess.upload_data('validation.csv', bucket=bucket, key_prefix=f'{prefix}/data/validation')

print(f"✅ Training data uploaded to: {train_s3_path}")
print(f"✅ Validation data uploaded to: {val_s3_path}")

## Summary

**What We Accomplished:**
- ✅ Initialized SageMaker session and S3 bucket
- ✅ Loaded and explored banking customer data
- ✅ Preprocessed data and engineered features
- ✅ Split data into training (80%) and validation (20%) sets
- ✅ Uploaded prepared data to S3 in XGBoost format

**Next Steps:**
- Proceed to Lab 2 to train the XGBoost model
- Configure hyperparameters for optimal performance
- Monitor training job progress